# Projeto Final — Fluxo Integrado de Preparação e Modelagem

Este notebook consolida as etapas de limpeza, análise exploratória e modelagem preditiva originalmente distribuídas nos notebooks `projeto_final.ipynb` e `notebooks/preparacao_ml.ipynb`. O objetivo é oferecer um fluxo único e organizado, cobrindo desde a extração dos dados no BigQuery até a avaliação do modelo e geração de previsões.


## Visão Geral do Notebook

1. **Configuração do ambiente e importações**
2. **Coleta dos dados no BigQuery**
3. **Análise exploratória dos dados**
4. **Preparação dos dados para machine learning**
5. **Treinamento e avaliação do modelo Random Forest**
6. **Simulações com perfis de pacientes e previsões no conjunto de teste**
7. **Recomendações e intervenções**


# 0. Configurações e Importações

Inicializamos as bibliotecas necessárias e garantimos que o ambiente possua as dependências para acessar o BigQuery e executar o fluxo de machine learning.


In [ ]:
!pip install google-cloud-bigquery --quiet
!pip install pandas scikit-learn matplotlib seaborn joblib --quiet

from google.cloud import bigquery
from google.colab import auth

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import joblib

%matplotlib inline


## 0.1 Autenticação no BigQuery

Autentique-se utilizando as credenciais do Google para acessar o projeto do BigQuery.


In [ ]:
auth.authenticate_user()


# 1. Coleta dos Dados no BigQuery

Consultamos as tabelas de treino e teste diretamente no projeto `t1engenhariadados` para garantir que utilizamos a versão mais atualizada dos dados.


In [ ]:
main_project = bigquery.Client(project="t1engenhariadados")


In [ ]:
    select_train_data = """"
SELECT *
FROM `t1engenhariadados.turma2_grupo9.train_data`
"""
    train = main_project.query(select_train_data).to_dataframe()

    select_test_data = """"
SELECT *
FROM `t1engenhariadados.turma2_grupo9.test_data`
"""
    test = main_project.query(select_test_data).to_dataframe()

    print(f"Dimensões do conjunto de treino: {train.shape}")
    print(f"Dimensões do conjunto de teste:  {test.shape}")


## 1.1 Preparação do DataFrame base

Manteremos uma cópia do conjunto de treino para exploração e pré-processamento, preservando os dados brutos retornados do BigQuery.


In [ ]:
df_raw = train.copy()
df_exploration = df_raw.copy()
df_raw.head()


# 2. Análise Exploratória dos Dados

A seguir investigamos a estrutura do dataset, valores ausentes e distribuições iniciais para orientar as decisões de limpeza e modelagem.


In [ ]:
df_exploration.info()


In [ ]:
df_exploration.describe(include="all")


## 2.1 Padrões e Relações

Exploramos correlações entre variáveis numéricas e a distribuição de `Stay` em relação a atributos categóricos relevantes.


In [ ]:
numeric_cols = df_exploration.select_dtypes(include=[np.number]).columns
plt.figure(figsize=(10, 6))
sns.heatmap(df_exploration[numeric_cols].corr(), annot=True, cmap="coolwarm")
plt.title("Correlação entre variáveis numéricas")
plt.show()


In [ ]:
    print("Value counts for Stay:")
    print(df_exploration["Stay"].value_counts().to_markdown())

    print("
Stay vs Hospital_type_code:")
    print(df_exploration.groupby("Hospital_type_code")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Department:")
    print(df_exploration.groupby("Department")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Ward_Type:")
    print(df_exploration.groupby("Ward_Type")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Severity of Illness:")
    print(df_exploration.groupby("Severity of Illness")["Stay"].value_counts(normalize=True).unstack().to_markdown())

    print("
Stay vs Age:")
    print(df_exploration.groupby("Age")["Stay"].value_counts(normalize=True).unstack().to_markdown())


In [ ]:
plt.figure(figsize=(10, 6))
sns.countplot(data=df_exploration, x="Stay", order=df_exploration["Stay"].value_counts().index)
plt.title("Distribuição da Duração da Estadia (Stay)")
plt.xlabel("Duração da Estadia")
plt.ylabel("Contagem")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

plt.figure(figsize=(12, 7))
sns.countplot(
    data=df_exploration,
    x="Severity of Illness",
    hue="Stay",
    palette="viridis",
    order=df_exploration["Severity of Illness"].value_counts().index,
)
plt.title("Duração da Estadia por Gravidade da Doença")
plt.xlabel("Gravidade da Doença")
plt.ylabel("Contagem")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


## 2.2 Fatores que influenciam a duração da internação

Com base nas análises realizadas, os seguintes fatores demonstraram maior influência na duração da internação (`Stay`):

### Gravidade da Doença (`Severity of Illness`)
Pacientes com gravidade classificada como *Extreme* ou *Moderate* apresentam uma tendência clara a ter estadias hospitalares mais longas, incluindo casos de `More than 100 Days`. Em contraste, pacientes com gravidade *Minor* geralmente têm estadias mais curtas (0-10 e 11-20 dias).

### Idade (`Age`)
Pacientes mais jovens (faixas etárias como 0-10 e 11-20 anos) tendem a ter períodos de internação mais curtos. Por outro lado, pacientes mais velhos (especialmente nas faixas de 81-90 e 91-100 anos) mostram uma proporção significativamente maior de estadias prolongadas, incluindo a categoria `More than 100 Days`.

### Tipo de Enfermaria (`Ward_Type`)
O tipo de enfermaria está associado a diferentes durações de estadia. Enfermarias do tipo `P` concentram estadias de até 30 dias, enquanto as dos tipos `S` e `T` apresentam maior proporção de estadias longas.

### Departamento (`Department`)
Departamentos como `radiotherapy` e `surgery` concentram estadias muito longas, sugerindo que procedimentos nesses contextos requerem um tempo de recuperação estendido. Já `anesthesia` apresenta maior proporção de estadias entre 11-20 dias.

### Tipo de Hospital (`Hospital_type_code`)
Hospitais do tipo `g` exibem ligeira predominância de estadias muito longas em comparação com outros tipos, enquanto `a` e `e` apresentam distribuição mais equilibrada entre estadias curtas e médias.


## 2.3 Interação entre idade e gravidade da doença

A interação entre a faixa etária (`Age`) e a gravidade da doença (`Severity of Illness`) revela padrões mais complexos na duração da internação (`Stay`).


### Principais insights da interação
1. **Idade e gravidade extrema:** pacientes mais velhos com gravidade *Extreme* apresentam as maiores proporções de estadias muito longas (`More than 100 Days`).
2. **Idade e gravidade menor:** em todas as idades, a gravidade *Minor* está associada a estadias curtas, com ligeiro aumento em pacientes mais velhos.
3. **Idade e gravidade moderada:** para gravidade *Moderate*, a duração da internação aumenta com a idade, reforçando a necessidade de monitoramento em faixas etárias avançadas.
4. **Padrão geral:** idade avançada combinada a gravidade moderada ou extrema é forte preditor de internações prolongadas.


# 3. Preparação dos Dados para Machine Learning

Aplicamos as etapas de limpeza do notebook `notebooks/preparacao_ml.ipynb`, garantindo que o dataset esteja adequado para o treinamento do modelo.


In [ ]:
def age_to_numeric(age_range):
    if isinstance(age_range, str) and "-" in age_range:
        low, high = age_range.split("-")
        return (int(low) + int(high)) / 2
    return age_range


In [ ]:
df_prepared = df_raw.copy()

# Tratamento de valores ausentes
bed_grade_mode = df_prepared["Bed Grade"].mode(dropna=True)[0]
df_prepared["Bed Grade"] = df_prepared["Bed Grade"].fillna(bed_grade_mode)

city_code_fill = 0
df_prepared["City_Code_Patient"] = df_prepared["City_Code_Patient"].fillna(city_code_fill)

# Conversão da idade para valores numéricos médios
df_prepared["Age"] = df_prepared["Age"].apply(age_to_numeric)

# Remoção de duplicatas
df_prepared = df_prepared.drop_duplicates()

# Tratamento de outliers via IQR
numeric_cols = df_prepared.select_dtypes(include=np.number).columns.tolist()
numeric_cols = [col for col in numeric_cols if col not in ["case_id", "patientid"]]

outlier_bounds = {}
for col in numeric_cols:
    Q1 = df_prepared[col].quantile(0.25)
    Q3 = df_prepared[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outlier_bounds[col] = (lower, upper)
    df_prepared = df_prepared[(df_prepared[col] >= lower) & (df_prepared[col] <= upper)]

# Transformação logarítmica em Admission_Deposit
if "Admission_Deposit" in df_prepared.columns:
    df_prepared["Admission_Deposit"] = np.log1p(df_prepared["Admission_Deposit"])

# Remoção de variáveis altamente correlacionadas
numeric_cols_clean = df_prepared.select_dtypes(include=np.number).columns.tolist()
corr_matrix = df_prepared[numeric_cols_clean].corr().abs()
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
columns_to_drop_corr = [column for column in upper.columns if any(upper[column] > 0.95)]
df_prepared = df_prepared.drop(columns=columns_to_drop_corr, errors="ignore")

# Agrupamento de categorias raras
frequent_categories = {}
cat_cols = df_prepared.select_dtypes(include="object").columns.tolist()
if "Stay" in cat_cols:
    cat_cols.remove("Stay")

for col in cat_cols:
    freq = df_prepared[col].value_counts(normalize=True)
    raras = freq[freq < 0.01].index
    allowed = set(freq[freq >= 0.01].index)
    if len(raras) > 0:
        df_prepared[col] = df_prepared[col].replace(list(raras), "OUTRA")
        allowed.add("OUTRA")
    frequent_categories[col] = allowed


In [ ]:
print(f"Dimensões após limpeza: {df_prepared.shape}")
df_prepared.head()


In [ ]:
preparation_artifacts = {
    "bed_grade_mode": bed_grade_mode,
    "city_code_fill": city_code_fill,
    "outlier_bounds": outlier_bounds,
    "columns_to_drop_corr": columns_to_drop_corr,
    "frequent_categories": frequent_categories,
}


In [ ]:
def prepare_dataset_for_inference(df, artifacts, drop_target=False):
    processed = df.copy()

    processed["Bed Grade"] = processed["Bed Grade"].fillna(artifacts["bed_grade_mode"])
    processed["City_Code_Patient"] = processed["City_Code_Patient"].fillna(artifacts["city_code_fill"])
    processed["Age"] = processed["Age"].apply(age_to_numeric)

    processed = processed.drop_duplicates()

    for col, (lower, upper) in artifacts["outlier_bounds"].items():
        if col in processed.columns:
            processed[col] = processed[col].clip(lower, upper)

    if "Admission_Deposit" in processed.columns:
        processed["Admission_Deposit"] = np.log1p(processed["Admission_Deposit"])

    processed = processed.drop(columns=artifacts["columns_to_drop_corr"], errors="ignore")

    for col, allowed in artifacts["frequent_categories"].items():
        if col in processed.columns:
            processed[col] = processed[col].where(processed[col].isin(allowed), "OUTRA")

    if drop_target and "Stay" in processed.columns:
        processed = processed.drop(columns=["Stay"])

    feature_dtypes = artifacts.get("feature_dtypes")
    feature_columns = artifacts.get("feature_columns")
    if feature_dtypes and feature_columns:
        for col, dtype_name in feature_dtypes.items():
            if col not in processed.columns:
                if "float" in dtype_name or "int" in dtype_name:
                    processed[col] = 0.0
                else:
                    processed[col] = "OUTRA"
        processed = processed.loc[:, feature_columns]

    return processed


# 4. Modelo Preditivo para Duração da Internação

Com o dataset limpo, estruturamos um pipeline que normaliza variáveis numéricas, aplica codificação one-hot em variáveis categóricas e treina um Random Forest Regressor.


In [ ]:
df_modeling = df_prepared.copy()

stay_mapping = {
    "0-10": 5,
    "11-20": 15,
    "21-30": 25,
    "31-40": 35,
    "41-50": 45,
    "51-60": 55,
    "61-70": 65,
    "71-80": 75,
    "81-90": 85,
    "91-100": 95,
    "More than 100 Days": 110,
}

stay_mapping_reverse = {v: k for k, v in stay_mapping.items()}

df_modeling["Stay_numeric"] = df_modeling["Stay"].map(stay_mapping)
df_modeling = df_modeling.dropna(subset=["Stay_numeric"])

id_columns = ["case_id", "patientid"]
X = df_modeling.drop(columns=id_columns + ["Stay", "Stay_numeric"], errors="ignore")
y = df_modeling["Stay_numeric"]

numeric_features = X.select_dtypes(include=np.number).columns.tolist()
categorical_features = X.select_dtypes(exclude=np.number).columns.tolist()

preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features),
    ]
)

regressor = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)

pipeline = Pipeline(steps=[("preprocessor", preprocessor), ("regressor", regressor)])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = mse ** 0.5
r2 = r2_score(y_test, y_pred)

print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R2): {r2:.2f}")

joblib.dump(pipeline, "random_forest_pipeline.pkl")

preparation_artifacts.update(
    {
        "feature_columns": X.columns.tolist(),
        "feature_dtypes": X.dtypes.astype(str).to_dict(),
        "id_columns": id_columns,
        "stay_mapping": stay_mapping,
        "stay_mapping_reverse": stay_mapping_reverse,
    }
)


## 4.1 Conclusão do modelo preditivo

O modelo Random Forest Regressor demonstrou capacidade de prever a duração da internação com boa aderência às métricas avaliadas. Os fatores identificados nas análises anteriores — gravidade da doença, idade, tipo de enfermaria, departamento e tipo de hospital — mostraram-se relevantes para a previsão. Para aprimoramentos futuros, recomenda-se explorar técnicas adicionais de engenharia de atributos e avaliar outros algoritmos de machine learning.


In [ ]:
def map_prediction_to_stay_category(prediction_value):
    closest_stay_numeric = min(
        preparation_artifacts["stay_mapping_reverse"].keys(),
        key=lambda k: abs(k - prediction_value),
    )
    return preparation_artifacts["stay_mapping_reverse"][closest_stay_numeric]


# 5. Simulações com Perfis de Pacientes

Utilizamos o pipeline treinado para estimar a duração da internação em perfis hipotéticos, facilitando a interpretação clínica do modelo.


In [ ]:
patient_profiles_data = [
    {
        "Hospital_code": 8,
        "Hospital_type_code": "a",
        "City_Code_Hospital": 1,
        "Hospital_region_code": "X",
        "Available Extra Rooms in Hospital": 4,
        "Department": "gynecology",
        "Ward_Type": "P",
        "Ward_Facility_Code": "F",
        "Bed Grade": 2.0,
        "City_Code_Patient": 7.0,
        "Type of Admission": "Emergency",
        "Severity of Illness": "Minor",
        "Visitors with Patient": 2,
        "Age": "0-10",
        "Admission_Deposit": 4000.0,
        "Profile_Name": "Jovem, Doença Leve",
    },
    {
        "Hospital_code": 26,
        "Hospital_type_code": "b",
        "City_Code_Hospital": 2,
        "Hospital_region_code": "Y",
        "Available Extra Rooms in Hospital": 2,
        "Department": "surgery",
        "Ward_Type": "S",
        "Ward_Facility_Code": "F",
        "Bed Grade": 3.0,
        "City_Code_Patient": 8.0,
        "Type of Admission": "Trauma",
        "Severity of Illness": "Extreme",
        "Visitors with Patient": 4,
        "Age": "81-90",
        "Admission_Deposit": 6000.0,
        "Profile_Name": "Idoso, Doença Extrema",
    },
    {
        "Hospital_code": 10,
        "Hospital_type_code": "e",
        "City_Code_Hospital": 3,
        "Hospital_region_code": "Z",
        "Available Extra Rooms in Hospital": 3,
        "Department": "radiotherapy",
        "Ward_Type": "T",
        "Ward_Facility_Code": "E",
        "Bed Grade": 4.0,
        "City_Code_Patient": 10.0,
        "Type of Admission": "Elective",
        "Severity of Illness": "Moderate",
        "Visitors with Patient": 3,
        "Age": "51-60",
        "Admission_Deposit": 5000.0,
        "Profile_Name": "Meia-idade, Doença Moderada",
    },
    {
        "Hospital_code": 1,
        "Hospital_type_code": "a",
        "City_Code_Hospital": 1,
        "Hospital_region_code": "X",
        "Available Extra Rooms in Hospital": 1,
        "Department": "gynecology",
        "Ward_Type": "P",
        "Ward_Facility_Code": "F",
        "Bed Grade": 1.0,
        "City_Code_Patient": 7.0,
        "Type of Admission": "Emergency",
        "Severity of Illness": "Extreme",
        "Visitors with Patient": 5,
        "Age": "0-10",
        "Admission_Deposit": 4500.0,
        "Profile_Name": "Jovem, Doença Extrema",
    },
    {
        "Hospital_code": 32,
        "Hospital_type_code": "f",
        "City_Code_Hospital": 6,
        "Hospital_region_code": "Y",
        "Available Extra Rooms in Hospital": 5,
        "Department": "anesthesia",
        "Ward_Type": "Q",
        "Ward_Facility_Code": "D",
        "Bed Grade": 3.0,
        "City_Code_Patient": 1.0,
        "Type of Admission": "Emergency",
        "Severity of Illness": "Minor",
        "Visitors with Patient": 1,
        "Age": "91-100",
        "Admission_Deposit": 3500.0,
        "Profile_Name": "Idoso, Doença Leve",
    },
]

patient_profiles_df = pd.DataFrame(patient_profiles_data)
profile_metadata = patient_profiles_df[["Profile_Name"]].copy()

patient_features = prepare_dataset_for_inference(
    patient_profiles_df.drop(columns=["Profile_Name"]),
    preparation_artifacts,
    drop_target=True,
)

profile_predictions_numeric = pipeline.predict(patient_features)

predicted_stay_categories = [
    map_prediction_to_stay_category(value) for value in profile_predictions_numeric
]

results_df = pd.DataFrame(
    {
        "Profile_Name": profile_metadata["Profile_Name"],
        "Predicted_Stay_Numeric": profile_predictions_numeric,
        "Predicted_Stay_Category": predicted_stay_categories,
    }
)

print("Previsões para os perfis de pacientes:")
print(results_df.to_markdown(index=False))


# 6. Previsão no Conjunto de Teste do BigQuery

Aplicamos a mesma preparação ao conjunto de teste para gerar estimativas da duração da internação em dados inéditos.


In [ ]:
test_features = prepare_dataset_for_inference(
    test,
    preparation_artifacts,
    drop_target=True,
)

test_predictions_numeric = pipeline.predict(test_features)
test_predicted_categories = [
    map_prediction_to_stay_category(value) for value in test_predictions_numeric
]

test_results = test.copy()
test_results["Stay_predicted"] = test_predictions_numeric
test_results["Stay_category"] = test_predicted_categories

test_results[["case_id", "Stay_predicted", "Stay_category"]].head()


# 7. Sugestões de Intervenções para Otimizar a Duração da Internação

Com base na análise aprofundada dos dados e nas previsões do modelo, é possível identificar perfis de pacientes com maior risco de internações prolongadas. As intervenções propostas visam otimizar a duração da estadia hospitalar, garantindo a qualidade do cuidado e a segurança do paciente, ao mesmo tempo em que se busca a eficiência operacional.

## 7.1 Intervenções focadas em pacientes idosos e/ou com doença extrema
Os perfis de pacientes idosos e aqueles com gravidade de doença extrema foram consistentemente associados às maiores durações de internação. A interação entre esses dois fatores é particularmente crítica. Para esses grupos, as intervenções devem ser multifacetadas:

* **Avaliação Geriátrica Abrangente (AGA) precoce:** Implementar a AGA nas primeiras 24-48 horas de internação para identificar riscos de complicações, declínio funcional, desnutrição, polifarmácia e síndromes geriátricas.
* **Equipes multidisciplinares especializadas:** Formar equipes dedicadas, incluindo geriatras, fisioterapeutas, terapeutas ocupacionais, nutricionistas, assistentes sociais e enfermeiros especializados.
* **Protocolos de alta otimizados:** Desenvolver protocolos específicos para pacientes de alto risco, com coordenação de cuidados pós-alta e educação da família.
* **Monitoramento intensivo e prevenção de complicações:** Aplicar pacotes de prevenção de complicações (p. ex. pneumonia associada à ventilação, infecções de corrente sanguínea, úlceras de pressão, delirium).
* **Revisão medicamentosa:** Realizar revisões farmacêuticas regulares para otimizar a medicação e reduzir eventos adversos.

## 7.2 Intervenções relacionadas ao departamento e tipo de enfermaria
Departamentos como radioterapia e cirurgia, e enfermarias dos tipos `S` e `T`, foram associados a estadias mais longas.

* **Otimização de fluxos de trabalho:** Agendar exames e procedimentos com eficiência, garantindo disponibilidade de leitos de recuperação e coordenação com equipes de reabilitação.
* **Unidades de transição de cuidados:** Criar unidades intermediárias para pacientes que não necessitam de cuidados intensivos, mas ainda não estão prontos para alta domiciliar.
* **Programas de reabilitação acelerada:** Implementar protocolos `ERAS (Enhanced Recovery After Surgery)` para reduzir o tempo de internação.

## 7.3 Intervenções gerais e tecnológicas
* **Uso de ferramentas preditivas:** Integrar o modelo desenvolvido aos sistemas de gestão hospitalar para identificar automaticamente pacientes com risco de estadias prolongadas.
* **Telemedicina e monitoramento remoto:** Oferecer acompanhamento contínuo fora do ambiente hospitalar, facilitando altas precoces com segurança.
* **Educação continuada da equipe:** Treinar profissionais sobre melhores práticas para gestão de pacientes com condições crônicas, cuidados paliativos e planejamento de alta.
* **Otimização da gestão de leitos:** Utilizar análises preditivas para reduzir o tempo de espera por leitos e otimizar o fluxo de pacientes.

## 7.4 Conclusão das intervenções
A otimização da duração da internação requer abordagem integrada. As intervenções sugeridas — focadas na personalização do cuidado, otimização de processos e adoção de tecnologias preditivas — podem melhorar a eficiência do sistema de saúde e os resultados para os pacientes. Recomenda-se acompanhamento e avaliação contínua para garantir a eficácia dessas estratégias.
